In [1]:

from nltk.corpus.reader import nltk
import gradio as gr
import numpy as np
import math
import os
import numpy as np
import matplotlib.pyplot as plt 

In [37]:
def combine_files(input_folder, output_file):
    with open(output_file, 'w') as output:
        for filename in os.listdir(input_folder):
            with open(input_folder+filename, 'r') as file:
                output.write(file.read())

id=[]
def read_chunks(input_file):
    with open(input_file, 'r') as file:
        content = file.readlines()
       # print(content[:5])
        chunks = []
        current_chunk = []
        oldline="*"*40
        b=False
        for line in content:
            if line.startswith('*' * 40):
                if current_chunk:
                    chunks.append(' '.join(current_chunk).replace('\n',' '))
                    current_chunk = []
                    b=False
            else:
                if ( line.startswith('Document') and oldline.startswith('*' * 40)):
                    b=True
                    line=line.split()
                    id.append(line[1])
                elif(b):
                    current_chunk.append(line)
            oldline=line
        if current_chunk:
            chunks.append(' '.join(current_chunk).replace('\n',' '))

    return chunks



input_folder = "Collection//"
output_file = 'LISA'
# combine_files(input_folder, output_file)

chunks=read_chunks(output_file)
id=np.array(id)
# _,indices=np.unique(id, axis=0, return_index=True)
# id=id[np.sort(indices)]
# indices=list(set(np.arange(1,6005))-set(id))
# chunks=np.delete(np.array(chunks), indices)


In [2]:
import string
MotsVides = nltk.corpus.stopwords.words('english')
folder_path="Collection//"
letter="[A-Za-z]"
sigle="[A-Z]\.(?:[A-Z]\.)+"
slash=f"{letter}+/{letter}+"
arobas="[\w]*@[\w]*"
pointsus="\.{3}"
realNumbers=f"\d+(?:[\.,]\d+)?(?:%|DA|{letter}*)?"
words="\w+"
casSpecial=f"(?:{letter}|\d+(?:[\.,]\d+))+(?:-(?:{letter}|\d+(?:[\.,]\d+)?)+)+"
casUnionWords=f"(?:{letter}|\d+(?:[\.,]\d+))+(?:-(?:{letter}|\d+(?:[\.,]\d+)?)+)*"
def textdocs(inverse,token,porter):
     liste_dic=[]
     textsdoc=[]
     for i in range(len(chunks)):
          text =chunks[i]
          if not token:
               text= text.split()  
          else:
               text=nltk.RegexpTokenizer('(?:[A-Za-z]\.)+|[A-Za-z]+[\-@]\d+(?:\.\d+)?|\d+[A-Za-z]+|\d+(?:[\.\,]\d+)?%?|\w+(?:[\-/]\w+)*').tokenize(text)
     
          if porter:
               stemmer = nltk.PorterStemmer()
          else:
               stemmer = nltk.LancasterStemmer()
         
          text = [word for word in text if word not in string.punctuation ] 
      
          TermesNormalisation= [stemmer.stem(terme) for terme in text if terme.lower() not in MotsVides]
          #TermesNormalisation.sort()
          liste_dic.append(nltk.FreqDist(TermesNormalisation))
     i=0
     for dic in liste_dic:
         
          maxfreq=max(dic.values())
          for terme,freq in dic.items():
               n=0
               for dico in liste_dic:
                    n+=(terme in dico)
               poid=(freq/maxfreq )* np.log10(len(liste_dic)/n+1)
               
               if inverse:
                    textsdoc.append(f'{terme} {id[i]} {freq} {poid}')
               else:
                    textsdoc.append(f'{id[i]} {terme} {freq} {poid}')
          i+=1
          #print(i)
     return textsdoc
# with open(folder_path+"..//resultats//DescripteurSplitLancaster.txt", "w") as fichier:
#       fichier.write('\n'.join(textdocs(0,0,0)))
# with open(folder_path+"..//resultats//DescripteurSplitPorter.txt", "w") as fichier:
#      fichier.write('\n'.join(textdocs(0,0,1)))
# with open(folder_path+"..//resultats//DescripteurTokenLancaster.txt", "w") as fichier:
#      fichier.write('\n'.join(textdocs(0,1,0)))
# with open(folder_path+"..//resultats//DescripteurTokenPorter.txt", "w") as fichier:
#      fichier.write('\n'.join(textdocs(0,1,1)))
# with open(folder_path+"..//resultats//InverseSplitLancaster.txt", "w") as fichier:
#      fichier.write('\n'.join(textdocs(1,0,0)))
# with open(folder_path+"..//resultats//InverseSplitPorter.txt", "w") as fichier:
#      fichier.write('\n'.join(textdocs(1,0,1)))
# with open(folder_path+"..//resultats//InverseTokenLancaster.txt", "w") as fichier:
#      fichier.write('\n'.join(textdocs(1,1,0)))
# with open(folder_path+"..//resultats//InverseTokenPorter.txt", "w") as fichier:
#      fichier.write('\n'.join(textdocs(1,1,1)))

In [3]:

a=[]
def visualisation(q,i,dpt,token,porter,methode,k,b):
     global a
     q_alg=q# a revoir 
     if dpt=="term per doc":
         
          if token=="Tokenisation":
               if porter=="Porter Stemmer":
                    fichier="resultats//DescripteurTokenPorter.txt"
               else:
                    fichier="resultats//DescripteurTokenLancaster.txt"
          else:
               if porter=="Porter Stemmer":
                    fichier="resultats//DescripteurSplitPorter.txt"
               else:
                    fichier="resultats//DescripteurSplitLancaster.txt"
     else:   
          if token=="Tokenisation":
               q=nltk.RegexpTokenizer(f'(?:[A-Za-z]\.)+|[A-Za-z]+[\-@]\d+(?:\.\d+)?|\d+[A-Za-z]+|\d+(?:[\.\,]\d+)?%?|\w+(?:[\-/]\w+)*').tokenize(q)
          
               if porter=="Porter Stemmer":
                    stemmer=nltk.PorterStemmer()
     
                    fichier="resultats//InverseTokenPorter.txt"
               else:
                  
                    stemmer=nltk.LancasterStemmer()
                    fichier="resultats//InverseTokenLancaster.txt"   
          else:
               q=q.split()
               if porter=="Porter Stemmer":
                    stemmer=nltk.PorterStemmer()
                    fichier="resultats//InverseSplitPorter.txt"
               else:
                  
                    stemmer=nltk.LancasterStemmer()
                    fichier="resultats//InverseSplitPorter.txt"     
          q = [word for word in q if word not in string.punctuation ] 
          q=[stemmer.stem(terme) for terme in q if terme.lower() not in MotsVides ] 
  
     if not dpt =="term per doc":  
          file = np.genfromtxt(fichier, delimiter=' ', dtype=object, encoding='utf-8',comments=None)
          file = np.array([[item.decode('utf-8') if isinstance(item, bytes) else item for item in row] for row in file])    
        
          if methode=="Produit Scalaire": 
               dict_poids={}
        

               for f in file:
                     if f[0] in q:
                         if f[1] in dict_poids:
                              dict_poids[f[1]]+=float(f[-1])
                         else:
                              dict_poids[f[1]]=float(f[-1])
              
               a=[[str(key),str(round(val,4))] for key,val in dict_poids.items()]
               
          elif methode=="Similarité Cosinus":
               dict_poids,dict_den={},{}
               for f in file:
                    if f[1] in dict_den:
                         dict_den[f[1]]+=float(f[-1])**2
                    else:
                         dict_den[f[1]]=float(f[-1])**2
                    if f[0] in q:
                         if f[1] in dict_poids:
                              dict_poids[f[1]]+=float(f[-1])
                         else:
                              dict_poids[f[1]]=float(f[-1])
               
               a=[[str(key),str(round(float(val)/(math.sqrt(len(q)*dict_den[key])),4))] for key,val in dict_poids.items()]

          elif methode=="Indice de Jaccard":
               dict_poids,dict_den={},{}
               for f in file:
                    if f[1] in dict_den:
                         dict_den[f[1]]+=float(f[-1])**2
                    else:
                         dict_den[f[1]]=float(f[-1])**2
                    if f[0] in q:
                         if f[1] in dict_poids:
                              dict_poids[f[1]]+=float(f[-1])
                         else:
                              dict_poids[f[1]]=float(f[-1])
               
               a=[[str(key),str(val/(len(q)+dict_den[key]-val))] for key,val in dict_poids.items()]
             
          elif methode=="BM25":
               dictf={}
               N=len(set(file[:,1]))
               k=float(k)
               b=float(b)
               nbrdoc={} 
               dl={} 
               for f in file:
                    if f[0] in q:
                         if f[0] in nbrdoc:
                              nbrdoc[f[0]]+=1
                         else:   
                              nbrdoc[f[0]]=1   
                    if f[1] in dl:
                         dl[f[1]]+=int(f[-2])
                    else:
                         dl[f[1]]=int(f[-2])
                   
               avdl=sum(dl.values())/N
               for f in file:
                    if f[0] in q:
                         freq=float(f[2])
                         ni=nbrdoc[f[0]]
                         if f[1] in dictf:
                              dictf[f[1]]+=((freq)/(k*((1-b)+b*(dl[f[1]]/(avdl)))+freq))*np.log10((N-ni+0.5)/(ni+0.5))
                         else:
                              dictf[f[1]]=((freq)/(k*((1-b)+b*(dl[f[1]]/(avdl)))+freq))*np.log10((N-ni+0.5)/(ni+0.5))

               a=[[str(key),str(round(val,4))] for key,val in dictf.items()]
               
          elif methode=="Alg":              


               mot_cle = r"(?:AND|OR)"
               terme = r"(?!(AND|OR|NOT))((?:[A-Za-z]\.)+|[A-Za-z]+[\-@]\d+(?:\.\d+)?|\d+[A-Za-z]+|\d+(?:[\.\,]\d+)?%?|\w+(?:[\-/]\w+)*)"
               termeG = r"(?:NOT )?" + terme
               v = fr"{termeG}(?: {mot_cle} {termeG})*"

               dico = {}
               if re.fullmatch(v, q_alg):
                    matches = re.finditer(termeG, q_alg)
                    matched_substrings = [match.group() for match in matches if match.group() not in {'ND', 'R'}]
                    indice_liste = [i for i, term in enumerate(matched_substrings) if "NOT" in term]
                    qk = [t for t in q_alg.split() if re.fullmatch(mot_cle, t)]

                    listedoc = set(f[1] for f in file)
                    file_array = np.array(file)

                    for doc in listedoc:
                         mot = np.array([not (np.any((file_array[:, 0] == q[i]) & (file_array[:, 1] == doc))) if i in indice_liste else np.any((file_array[:, 0] == q[i]) & (file_array[:, 1] == doc)) for i in range(len(q))])

                         if svd(mot, qk):
                              dico[doc] = 1

                    a = [[str(key), str(val)] for key, val in dico.items()]
          elif methode=="Coef de Dice":
               dict_poids,dict_den={},{}
               for f in file:
                    if f[1] in dict_den:
                         dict_den[f[1]]+=float(f[-1])**2
                    else:
                         dict_den[f[1]]=float(f[-1])**2
                    if f[0] in q:
                         if f[1] in dict_poids:
                              dict_poids[f[1]]+=float(f[-1])
                         else:
                              dict_poids[f[1]]=float(f[-1])
               
               a=[[str(key),str(round(2*val/(len(q)+dict_den[key]),4))] for key,val in dict_poids.items()]
               
          
     else:
          resultat=[]
          with open(fichier,'r') as filee:
                    for line in filee:
                         tabLigne=line.split()
                         if tabLigne[0]==q:
                              resultat.append(tabLigne)   
                 
                    
          return [resultat,None]+[[],[],[],[],[]]
     
     if not int(i)==0:     
          eva=evaluation(i)
          curv=p_r_curve_plot(eva[1],eva[2])
     else:
          eva=([[],[],[],[],[]],0,0)
          curv=None
    
     liste=[a,curv]+eva[0]
     return liste

def svd(termes,key_liste):
     
     if len(key_liste)==1:
          if key_liste[0]=="OR":
               return termes[0] or termes[1]
          elif key_liste[0]=="AND":
               return termes[0] and termes[1]
     elif len(key_liste)==0:
          return termes[0]
     else:
          avant=[]
          apres=[]
          reversed_list = key_liste[::-1]
          if "OR" in key_liste:#si and alors avant apres and sinon avant apres 1st or
          
                j=len(key_liste)-1-reversed_list.index("OR")#dernier
                i=len(key_liste)
               
                while(i>=0):
                    if(i>j):
                        apres.append(termes[i]) 
                    elif(i<=j):
                        avant.append(termes[i])
                    i-=1
          elif "AND"in key_liste:
                j=len(key_liste)-1-reversed_list.index("AND")#dernier
                i=len(key_liste)
                while(i>=0):
                    if(i>j):
                        apres.append(termes[i]) 
                    elif(i<=j):
                        avant.append(termes[i])
                    i-=1
          
          if type(key_liste[:j]) is bool:
               liste=[key_liste[:j]]
          else:
               liste=key_liste[:j]
          if type(key_liste[j+1:]) is bool:
               liste2=[key_liste[j+1:]]
          else:
               liste2=key_liste[j+1:]
          #print(avant,liste,apres,liste2,[key_liste[j]])
     
          return svd([svd(avant,liste[::-1]),svd(apres,liste2[::-1])] ,[key_liste[j]])
 
def p_r_curve_plot(p_values,r_values):
     plot = plt.figure()
     plt.plot(p_values, r_values)
     plt.xlabel(f'recall')
     plt.ylabel('precision')
     plt.title(f'curve recall / precision')
     plot.savefig("cruve.png")
     plt.close(plot)
     plot = ["cruve.png"]
     return plot
def evaluation(ii):
     nom_fichier="pertinence//LISARJ.NUM"
     chiffres=[]
     Judgement=np.empty((0,2),dtype=int)
     with open(nom_fichier, 'r') as fichier:
          for ligne in fichier:
               chiffres.extend(nltk.RegexpTokenizer("\d+").tokenize(ligne))
          i=0
          while i < len(chiffres):
               x=int(chiffres[i])
               i+=2
               y=np.array(chiffres[i:i+int(chiffres[i-1])]).astype(int)
               for el in y:
                    Judgement=np.vstack((Judgement,[x,el]))
               i=i+int(chiffres[i-1])

     r=np.array(a).astype(float)
     
     r = r[r[:, 1].argsort()[::-1]]
 

     rep=[l[1] for l in Judgement if l[0]==(int(ii)) ]
     
     precision=len(set(r[:,0]) & set(rep))/len(r[:,0])

     P5=len(set(r[:5,0]) & set(rep))/5
     P10=len(set(r[:10,0]) & set(rep))/10

     rappel=len(set(r[:,0]) & set(rep))/len(rep)
     F_score=(2*precision*rappel)/(rappel+precision)

     p_values,r_values=[],[]
     pertinent=0
     p_values_inter,r_values_inter=[],np.arange(0,1.1,0.1)
     for i in range(1,r.shape[0]+1):
          if r[i-1,0] in rep :
               pertinent+=1
          p_values.append(pertinent/i)
          r_values.append(pertinent/len(rep))
     
     p_values=np.array(p_values)
     for i in range(len(r_values_inter)):
          indices=[j for j in  range(len(r_values)) if  r_values[j]>=r_values_inter[i]]
          if len(indices)==0:
               p_values_inter.append(0)
          else:
               p_values_inter.append(np.max(p_values[indices]))
     print(p_values_inter)
     print("kkk")
     return ([str(precision),str(P5),str(P10),str(rappel),str(F_score)],list(r_values_inter),p_values_inter)

In [4]:
def changing(i):
     i=int(i)
     queryfile="pertinence//LISA.QUE"
     with open(queryfile,'r') as file :
          lignes,query=[],[]
          oldline="#"
          for line in file:
               if line.strip().endswith("#"):
                    lignes.append(line.replace('\n',' ')[:-2])
                    query.append(' '.join(lignes))
                    lignes=[]
               else:
                    if not oldline.strip().endswith("#"):
                         lignes.append(line.replace('\n',' '))
               
               oldline=line
     queries = np.array(query)
     if i>=1 and i<=len(queries):
          return queries[i-1]
     else:
          return ''
def update_visibility(selected):
          if selected == "BM25":
               return {bm: gr.Row(visible=True)}
          else:
               return {bm: gr.Row(visible=False)}
def update_visibility2(selected):
     if selected == "term per doc":
          return {rsd: gr.Row(visible=False)}
     else:
          return {rsd: gr.Row(visible=True)}
with gr.Blocks(theme=gr.themes.Soft()) as block:
     with gr.Row():
          inputs = [
          gr.components.Text(label="QUERY",scale=3),gr.components.Number(label="DATASET QUERY") 
          ]
          inputs[1].change(changing,inputs[1],inputs[0])
     with gr.Row():
          inputs.extend([
                 gr.Radio(
          ["doc per term", "term per doc"], label="RECHERCHE PAR DOCUMENT OU PAR TERME:"
     ),
          gr.Radio(
               ["Tokenisation", "Split"], label="TECHNIQUE DE TOKENISATION:"
          ),
          gr.Radio(
          ["Lancaster", "Porter Stemmer"], label="TECHNIQUE DE NORMALISATION"
     )
        ])
     with gr.Row(visible=True) as rsd:
          inputs.extend([
          gr.Radio(
          ["Produit Scalaire","Similarité Cosinus","Indice de Jaccard","BM25","Alg","Coef de Dice"], label="METHODE D''APPARAIMENT "
     )])
     inputs[2].change(update_visibility2, inputs=inputs[2], outputs=[rsd])
     with gr.Row(visible=False) as bm:  
          inputs.extend([
     gr.components.Number(label="k"),
     gr.components.Number(label="B")])
     inputs[5].change(update_visibility, inputs=inputs[5], outputs=[bm])
     with gr.Row():
          outputs= [gr.Dataframe(row_count = (1, "dynamic"), col_count=(4,"static"),  headers=["terme","document", "Frequence", "Poids"],label="Output Data", interactive=1)]
     with gr.Column():
          btn=gr.Button("Lancer") 
     with gr.Row():
          eval_output=[gr.Textbox(label="PRECISION"), gr.Textbox(label="P5"), gr.Textbox(label="P10"), gr.Textbox(label="RECALL"), gr.Textbox(label="F-SCORE")]
     with gr.Column():
          graph=[gr.Gallery(label="GRAPH", columns=(1,2),height=500)]
          btn.click(fn=visualisation, inputs=inputs, outputs=outputs+graph+eval_output)
block.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


IMPORTANT: You are using gradio version 4.19.1, however version 4.29.0 is available, please upgrade.
--------
